# Imports, read csv, functions definitions

In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from utils_rnn import add_xy_and_deltas
from utils_rnn import split_train_val_test
from utils_rnn import create_sequences
from utils_rnn import autoregressive_predict
from utils_rnn import reconstruct_positions
from utils_rnn import plot_input_and_predictions
from utils_rnn import folium_plot_trip_with_prediction
from utils_rnn import mass_xy_to_latlon
from utils_rnn import compute_errors
from utils_rnn import haversine

# --------------------------
# Configurable parameters
# --------------------------
CSV_PATH = "data/ais_data_5min_clean.csv"   # <-- replace with your file
INPUT_FEATURES = ["dx", "dy"]  # easy to change later
OUTPUT_FEATURES = ["dx", "dy"] # identical for autoregression
TEST_SIZE = 0.2
VAL_SIZE = 0.1
RANDOM_STATE = 42

# Train Validation Test split

In [ ]:
from sklearn.preprocessing import StandardScaler

# 1. Load data
df = pd.read_csv(CSV_PATH)

# Expect columns: MMSI, segment, lat, lon, timestamp (optional)
print("Loaded data:", df.shape)

# 2. Convert lat/lon to x/y and compute deltas
df = add_xy_and_deltas(df)

# 3. Split into train/val/test
train_df, val_df, test_df = split_train_val_test(df)

print("Train size:", train_df.shape)
print("Val size:", val_df.shape)
print("Test size:", test_df.shape)

SEQ_LEN = 10
X_train, y_train, _         = create_sequences(train_df, INPUT_FEATURES, OUTPUT_FEATURES, seq_len=SEQ_LEN)
X_val, y_val, _             = create_sequences(val_df, INPUT_FEATURES, OUTPUT_FEATURES, seq_len=SEQ_LEN)
X_test, y_test, test_meta   = create_sequences(test_df, INPUT_FEATURES, OUTPUT_FEATURES, seq_len=SEQ_LEN)

num_sequences, seq_len, num_features = X_train.shape
X_train_flat = X_train.reshape(-1, num_features)

# WITHOUT TRANSFORM IT WORKS BEST
""" # Fit scaler on training data only
scaler = StandardScaler()
scaler.fit(X_train_flat)

# Transform all sets
X_train = scaler.transform(X_train_flat).reshape(num_sequences, seq_len, num_features)

# Validation
X_val = scaler.transform(X_val_original.reshape(-1, num_features)).reshape(X_val_original.shape)
X_test = scaler.transform(X_test_original.reshape(-1, num_features)).reshape(X_test_original.shape)

# Targets (Y) also normalized with same scaler
y_train = scaler.transform(y_train_original)
y_val   = scaler.transform(y_val_original)
y_test  = scaler.transform(y_test_original) """



print("Feature shapes:", X_train.shape, y_train.shape)


# RNN

In [ ]:
%matplotlib inline
# Import deep learning libraries
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import GRU, Dense, Dropout, BatchNormalization
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau, ModelCheckpoint
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import numpy as np

print(f"TensorFlow version: {tf.__version__}")
print(f"GPU Available: {tf.config.list_physical_devices('GPU')}")


In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import SimpleRNN, Dense, Dropout, BatchNormalization
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.regularizers import l2

def build_rnn_model(input_shape, output_dim):
    model = Sequential()

    # Stacked RNN layers
    model.add(SimpleRNN(2*128, return_sequences=True, activation='relu', input_shape=input_shape))
    model.add(Dropout(0.3))
    model.add(SimpleRNN(2*64, return_sequences=False, activation='relu'))

    # Dense layers for nonlinear mapping
    model.add(BatchNormalization())
    model.add(Dense(64, activation='relu'))
    model.add(Dropout(0.3))

    # Final regression output
    model.add(Dense(output_dim, kernel_regularizer=l2(1e-4)))

    # Compile
    model.compile(
        optimizer=Adam(learning_rate=1e-4),
        loss='mse',
        metrics=['mae']
    )
    
    return model


In [ ]:
n_timesteps = X_train.shape[1]
n_features = X_train.shape[2]
n_targets = y_train.shape[1]

model = build_rnn_model(
    input_shape=(n_timesteps, n_features),
    output_dim=n_targets
)

model.summary()

In [ ]:
import signal
from tensorflow.keras.callbacks import Callback

class GracefulInterrupt(Callback):
    def __init__(self):
        super().__init__()
        self.stop_training = False
        signal.signal(signal.SIGINT, self.handle_sigint)

    def handle_sigint(self, signum, frame):
        print("\nSIGINT received: Training will stop after this epoch.\n")
        self.stop_training = True

    def on_epoch_end(self, epoch, logs=None):
        if self.stop_training:
            print(f"Stopping at epoch {epoch+1}.")
            self.model.stop_training = True


callbacks = [
    EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True),
    ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=5),
    GracefulInterrupt()
]

history = model.fit(
    X_train, y_train,
    validation_data=(X_val, y_val),
    epochs=100,
    batch_size=64,
    callbacks=callbacks,
    verbose=1
)

# Evaluation

In [ ]:
horizon = 10
all_stats = []

# container for per-step errors across all sequences
step_errors = [[] for _ in range(horizon)]

for seq_id in range(len(X_test)):
    if seq_id > 1000:
        break
    if seq_id % 50 == 0:
        print(seq_id, "out of", len(X_test))
    
    # --- Predict horizon steps ---
    preds = autoregressive_predict(model, X_test[seq_id], horizon)
    # --- Reconstruct positions ---
    start_idx = test_meta[seq_id]["end_index"]  # last input row
    start_xy = df.loc[start_idx, ["x","y"]].values
    pred_positions_xy = reconstruct_positions(preds, start_xy)[1:] # I want only the predictions, not the "starting" point (which is the last true)
    
    # --- Convert to lat/lon ---
    pred_positions_latlon = mass_xy_to_latlon(pred_positions_xy)
    target_indices = [test_meta[seq_id]["target_index"] + k for k in range(horizon)]
    true_positions_latlon = df.loc[target_indices, ["Latitude","Longtitude"]].values
    
    # --- Compute stats per step ---
    # compute haversine distance for each step
    for step in range(horizon):
        err = haversine(tuple(true_positions_latlon[step]), tuple(pred_positions_latlon[step]))
        step_errors[step].append(err)

# --- Aggregate error per step ---
print("\nStep-wise error statistics (meters):")
for step in range(horizon):
    errs = np.array(step_errors[step])
    mean_e = np.mean(errs)
    std_e = np.std(errs)
    med_e = np.median(errs)
    print(f"Step {step+1}: mean={mean_e:.2f}, std={std_e:.2f}, median={med_e:.2f}, n={len(errs)}")

